In [95]:
import sys
from azureml.core import Workspace, Experiment
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from azureml.core.runconfig import DataReferenceConfiguration

In [ ]:
!{sys.executable[:-6]}/pip install --upgrade azureml-sdk[notebooks,contrib]

In [103]:
import sys
!conda env export --no-builds --prefix {sys.executable[:-10]} > environment.yml

In [104]:
cat environment.yml

name: null
channels:
  - pytorch
  - conda-forge
  - defaults
dependencies:
  - appnope=0.1.0
  - attrs=18.2.0
  - backcall=0.1.0
  - bleach=3.1.0
  - ca-certificates=2018.11.29
  - certifi=2018.11.29
  - cffi=1.11.5
  - decorator=4.3.2
  - entrypoints=0.3
  - freetype=2.9.1
  - ipykernel=5.1.0
  - ipython=7.2.0
  - ipython_genutils=0.2.0
  - ipywidgets=7.4.2
  - jedi=0.13.2
  - jinja2=2.10
  - jpeg=9c
  - jsonschema=3.0.0a3
  - jupyter_client=5.2.4
  - jupyter_core=4.4.0
  - jupyterlab=0.35.4
  - jupyterlab_server=0.2.0
  - libcxx=7.0.0
  - libffi=3.2.1
  - libpng=1.6.36
  - libsodium=1.0.16
  - libtiff=4.0.10
  - llvm-meta=7.0.0
  - markupsafe=1.1.0
  - mistune=0.8.4
  - mkl_fft=1.0.10
  - mkl_random=1.0.2
  - nb_conda=2.2.1
  - nb_conda_kernels=2.2.0
  - nbconvert=5.3.1
  - nbformat=4.4.0
  - ncurses=6.1
  - ninja=1.8.2
  - notebook=5.7.4
  - olefile=0.46
  - openssl=1.1.1a
  - pandoc=2.5
  - pandocfilters=1.4.2
  - parso=0.3.2
  - pexpect=4.6.0
  - pickleshare=0.7.5
  - pillow=5.4.

In [ ]:
ws = Workspace.from_config()

In [ ]:
ds = ws.get_default_datastore()

In [ ]:
data = ds.upload('data', 'forecasting/data')

In [ ]:
experiment = Experiment(ws,"forecasting")

In [ ]:
cluster = ws.compute_targets['newcluster']
cluster.get_status().serialize()

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
#from azureml.core.runconfig import DEFAULT_CPU_IMAGE
#from azureml.core.script_run_config import ScriptRunConfig

run_config = RunConfiguration()
#run_config.environment.docker.enabled = True
#run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
#run_config.environment.python.user_managed_dependencies = False
#run_config.auto_prepare_environment = True
run_config.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='environment.yml')

#script_run_config = ScriptRunConfig(source_directory='../../',\n",
#                                    script='scripts/notebook_launcher.py',\n",
#                                    arguments=['--notebook_path', 'notebooks/00_quick_start/fastai_movielens.ipynb', 'MOVIELENS_DATA_SIZE', '100k'],\n",
#                                    run_config=run_config)\n",

est = Estimator(source_directory='.', 
                compute_target=cluster, 
                entry_script='1_time_series_arima.ipynb',
                pip_packages=['numpy', 'pandas', 'sklearn'])

run = experiment.submit(est)
run


In [55]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [59]:
from azureml.contrib.notebook.notebook_run_config import NotebookRunConfig

In [107]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create a new runconfig object
rc = RunConfiguration()

# Use the cpu_cluster you created above. 
rc.target = cluster

# Enable Docker
rc.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
rc.environment.docker.base_image = DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
rc.environment.python.user_managed_dependencies = False

# Auto-prepare the Docker image when used for execution (if it is not already prepared)
rc.auto_prepare_environment = True

# Specify CondaDependencies obj, add necessary packages
rc.environment.python.conda_dependencies = \
    CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas', 'matplotlib'], pip_packages=['statsmodels'])

#rc.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='environment.yml')

rc.data_references = {'data':DataReferenceConfiguration(datastore_name='workspaceblobstore', path_on_datastore=data.path_on_datastore)}

In [110]:
nbrc = NotebookRunConfig(source_directory='.', parameters=['data_path','$AZ_DATASTORE_data'], notebook='1_time_series_arima.ipynb', run_config=rc)

In [111]:
run = experiment.submit(nbrc)
run

Experiment,Id,Type,Status,Details Page,Docs Page
forecasting,forecasting_1555820850_0c061cf1,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


In [102]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [82]:
print(rc.environment.python.conda_dependencies.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.23.*
  - papermill<1.0
  - ipykernel
- sklearn
- numpy
- pandas
- matplotlib



In [98]:
ws.get_default_datastore().name

'workspaceblobstore'

In [122]:

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [119]:
??AmlComputeConfiguration

In [121]:
from azureml.core import ComputeTarget

In [123]:
foo = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

In [124]:
foo.__class__

azureml.core.compute.amlcompute.AmlComputeProvisioningConfiguration